In [1]:
import os
import s3fs
import pandas as pd

os.chdir("../")

from src.core.configs import settings, fs
from src.dataloader.loader import CustomImageDataset, MaskDataloaders

In [2]:
settings

GlobalSettings(s3_endpointUrl='https://minio.lab.sspcloud.fr', s3_key='XQ1UXZ3DRKNHS34K6FPD', s3_secret='f+L3LAmOHnm92jHIOvw0mCm+hc33PgJ9chBr5gZ+', s3_token='eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJYUTFVWFozRFJLTkhTMzRLNkZQRCIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzAxNjMyNDg3LCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6Im5pY2tqb2ZyZWluLnRlZG9uemVAZW5zYWUuZnIiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzAxOTU1MTI1LCJmYW1pbHlfbmFtZSI6Ik5JQ0sgSk9GUkVJTiIsImdpdmVuX25hbWUiOiJURURPTlpFIiwiZ3JvdXBzIjpbXSwiaWF0IjoxNzAxODY4NzI1LCJpc3MiOiJodHRwczovL2F1dGgubGFiLnNzcGNsb3VkLmZyL2F1dGgvcmVhbG1zL3NzcGNsb3VkIiwianRpIjoiMTg1ODM2MTMtZWQ0Zi00YTA3LWJlODAtOGI3ZjA0NjM2YTRlIiwibmFtZSI6IlRFRE9OWkUgTklDSyBKT0ZSRUlOIiwicG9saWN5Ijoic3Rzb25seSIsInByZWZlcnJlZF91c2VybmFtZSI6InR0ZWRvbnplIiwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLXNzcGNsb3VkIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6e

In [23]:
train=MaskDataloaders(settings.s3_prefix, [0.7, 0.2, 0.1]).getDataloaders()[0]

In [16]:
settings

GlobalSettings(s3_endpointUrl=None, s3_key=None, s3_secret=None, s3_token=None, s3_prefix=None)

In [4]:
from torch import flatten,dropout,softmax


In [5]:
!ls

data  Dockerfile  LICENSE  main.py  notebooks  README.md  requirements.txt  src


In [94]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, number_class):
        super().__init__() 
        self.number_class = number_class
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=7,padding='same')
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 9, 7 ,padding='same')
        self.conv3 = nn.Conv2d(9, 12, 7,padding='same')
        self.fc1 = nn.Linear(12*16*16, 100)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(100, 30)
        self.fc3 = nn.Linear(30, number_class)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x




In [95]:
import torch
import torch.nn as nn
import torch.optim as optim


class BaseModel():
    def __init__(self, model, lr=1e-3, momentum=0.9):
        self.model = model
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.SGD(
            model.parameters(), 
            lr=lr,
            momentum=momentum
            )

    def fit(self, dataloader, n_epoch=10):
        for epoch in range(n_epoch):
            running_loss = 0.0
            for i, data in enumerate(dataloader):
                inputs, labels = data
                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                running_loss += loss.item()
                if i % 10 == 9:
                    print(f'''epoch: {epoch},batch number {i+1} 
                    .Loss:{running_loss/10 :.3f}''')
                    running_loss = 0

    def save_model(self, path):
        torch.save(self.model.state_dict(), path)

    def load_model(self, path):
        self.model.load_dict(path)

    
        








        




In [96]:
net=Net(2)
model=BaseModel(net)
model.fit(train)

epoch: 0,batch number 10 
                    .Loss:0.694
epoch: 0,batch number 20 
                    .Loss:0.693
epoch: 1,batch number 10 
                    .Loss:0.693
epoch: 1,batch number 20 
                    .Loss:0.693
epoch: 2,batch number 10 
                    .Loss:0.693
epoch: 2,batch number 20 
                    .Loss:0.693
epoch: 3,batch number 10 
                    .Loss:0.693
epoch: 3,batch number 20 
                    .Loss:0.693


KeyboardInterrupt: 

In [97]:
torch.cuda.device_count()

0

Files already downloaded and verified


torch.Size([4, 3, 32, 32])

Net(
  (conv1): Conv2d(3, 6, kernel_size=(7, 7), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 9, kernel_size=(7, 7), stride=(1, 1))
  (conv3): Conv2d(9, 12, kernel_size=(7, 7), stride=(1, 1))
  (fc1): Linear(in_features=3072, out_features=100, bias=True)
  (fc2): Linear(in_features=50, out_features=30, bias=True)
  (fc3): Linear(in_features=30, out_features=2, bias=True)
)
